In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris

In [36]:
iris = load_iris()

data, target = iris["data"], iris["target"].reshape(1, -1)
print(f"data의 type : {type(data)},  data의 shape : {data.shape}")
print(f"target의 type : {type(target)},  target의 shape : {target.shape}")

data의 type : <class 'numpy.ndarray'>,  data의 shape : (150, 4)
target의 type : <class 'numpy.ndarray'>,  target의 shape : (1, 150)


In [37]:
print(np.max(data), np.min(data))

# Normalize data to keep our gradients manageable
data = data / 7.9
print(np.max(data), np.min(data))

7.9 0.1
1.0 0.012658227848101266


In [38]:
np.random.seed(2023)
shuffle_idx = np.random.permutation(data.shape[0])
print(shuffle_idx)
print()

m = 120

train_x, test_x = data[shuffle_idx[:m], :], data[shuffle_idx[m:]]
train_y, test_y = target[:, shuffle_idx[:120]], target[:, shuffle_idx[120:]]

print(f"train_x의 shape : {train_x.shape},  train_y의 shape : {train_y.shape}")
print(f"test_x의 shape  : {test_x.shape},  test_y의 shape  : {test_y.shape} ")

[128  74  99 107  76 113  84  96   4  62  30  86  41 137  17 120  32  57
  35  16  94   2 122  50  23  14  21 135  81  48   9  38  18 119  98 109
  12  89 147  20  64 103  82  90  85 126  59  75  92  27  44 106   8  33
  73 141  95  68  42   0  61 129 121  13  56 124  80  79  67 145  24  54
 132  60 125 139  66  26  93  97  36 115 136  37  70  78  46 138   6  55
  83 118  10   5  43 144 105  40  19  51  69  31   1  11 108 123 102 104
 133 117 149  91 142 140  34 148 146  45   7 130 143  71  65 100  47  72
  88 127 111  15  53 134 112 114  49  29  63 110  28  58 131  39  77  22
 101  52 116   3  25  87]

train_x의 shape : (120, 4),  train_y의 shape : (1, 120)
test_x의 shape  : (30, 4),  test_y의 shape  : (1, 30) 


In [39]:
train_label, tr_label_num = np.unique(train_y, return_counts=True)
test_label, te_label_num = np.unique(test_y, return_counts=True)

train_y_dict = {}
for label, num in zip(train_label, tr_label_num):
    train_y_dict[label] = num
    
test_y_dict = {}
for label, num in zip(test_label, te_label_num):
    test_y_dict[label] = num
    
print(f"train_y의 label분포 : {train_y_dict}")
print(f"test_y의 label분포  : {test_y_dict}")

train_y의 label분포 : {0: 41, 1: 40, 2: 39}
test_y의 label분포  : {0: 9, 1: 10, 2: 11}


In [40]:
train_y[:, np.where(train_y == 0)[1]]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [41]:
# To build a zero-classifier, map target value of 0 into 1 and map all other target values into 0
train_y_new = np.zeros(train_y.shape)
train_y_new[:, np.where(train_y == 0)[1]] = 1
train_y = train_y_new
print(f"train_y의 분포 : {np.unique(train_y, return_counts=True)}")

test_y_new = np.zeros(test_y.shape)
test_y_new[:, np.where(test_y == 0)[1]] = 1
test_y = test_y_new
print(f"test_y의 분포 : {np.unique(test_y, return_counts=True)}")

train_y의 분포 : (array([0., 1.]), array([79, 41], dtype=int64))
test_y의 분포 : (array([0., 1.]), array([21,  9], dtype=int64))


In [42]:
# We'll split data into train and test set. We also transpose data to keep each example as a column.
x_train, x_test = train_x.T, test_x.T
y_train, y_test = train_y, test_y

print(f"x_train의 shape : {x_train.shape},   y_train의 shape : {y_train.shape}")
print(f"x_test의  shape : {x_test.shape},   y_test의 shape :  {y_test.shape}")

x_train의 shape : (4, 120),   y_train의 shape : (1, 120)
x_test의  shape : (4, 30),   y_test의 shape :  (1, 30)


In [43]:
np.unique(y_train, return_counts=True), np.unique(y_test, return_counts=True)

((array([0., 1.]), array([79, 41], dtype=int64)),
 (array([0., 1.]), array([21,  9], dtype=int64)))

In [44]:
def sigmoid(z):
    s = 1.0 / (1.0 + np.exp(-z))
    return s

In [45]:
def compute_loss(y, y_hat):
    m = y.shape[1]
    loss = -(1/m) * np.sum(np.multiply(y, np.log(y_hat)) + np.multiply((1-y), np.log(1-y_hat)))
    return loss

In [46]:
m = 60000
learning_rate = 1

X, Y = x_train, y_train
n_x, m = X.shape[0], X.shape[1]

W = np.random.randn(n_x, 1) * 0.01
b = np.zeros((1, 1))

Z = np.matmul(W.T, X) + b
A = sigmoid(Z)

cost = compute_loss(Y, A)

dW = (1/m) * np.matmul(X, (Y-A).T)
db = (1/m) * np.sum(Y-A)

W -= learning_rate * dW
b -= learning_rate * db

print(f"W의 shape : {W.shape}\nX의 shpae : {X.shape}\nb의 shape : {b.shape}\nZ의 shape : {Z.shape}\nA의 shape : {A.shape}\ncost의 value : {cost}")
print(f"dW의 shape : {dW.shape}\ndb의 shape : {db.shape}, Y의 shape {Y.shape}")

W의 shape : (4, 1)
X의 shpae : (4, 120)
b의 shape : (1, 1)
Z의 shape : (1, 120)
A의 shape : (1, 120)
cost의 value : 0.6924396879186446
dW의 shape : (4, 1)
db의 shape : (), Y의 shape (1, 120)


In [47]:
learning_rate = 1

X, Y = x_train, y_train
n_x, m = X.shape[0], X.shape[1]

W = np.random.randn(n_x, 1) * 0.01
b = np.zeros((1, 1))

for i in range(2001):
    Z = np.matmul(W.T, X) + b
    A = sigmoid(Z)
    
    cost = compute_loss(Y, A)
    
    dW = (1/m) * np.matmul(X, (A-Y).T)
    db = (1/m) * np.sum(A - Y)

    W -= learning_rate * dW
    b -= learning_rate * db
    
    if i % 100 == 0:
        print(f"Epoch {i:4d}의 cost : {cost}")
print()       
print(f"Final cost : {cost}")

Epoch    0의 cost : 0.6941048511301275
Epoch  100의 cost : 0.221563758385273
Epoch  200의 cost : 0.13348031046976716
Epoch  300의 cost : 0.09649635187351857
Epoch  400의 cost : 0.07611181540319155
Epoch  500의 cost : 0.06314600703797787
Epoch  600의 cost : 0.05414073695256521
Epoch  700의 cost : 0.04750330408643421
Epoch  800의 cost : 0.042397128994964624
Epoch  900의 cost : 0.03834000083922004
Epoch 1000의 cost : 0.03503402742844641
Epoch 1100의 cost : 0.032285027694207255
Epoch 1200의 cost : 0.029960879150186234
Epoch 1300의 cost : 0.027968478840440385
Epoch 1400의 cost : 0.026240277458760685
Epoch 1500의 cost : 0.024726041393971537
Epoch 1600의 cost : 0.023387615741317327
Epoch 1700의 cost : 0.022195484120121706
Epoch 1800의 cost : 0.021126443892805342
Epoch 1900의 cost : 0.020161995818563795
Epoch 2000의 cost : 0.019287204036204434

Final cost : 0.019287204036204434


In [48]:
# To assess how well training performs, we'll see some accuracy measure
from sklearn.metrics import classification_report, confusion_matrix

Z = np.matmul(W.T, x_test) + b
A = sigmoid(Z)

predictions = (A > 0.5)[0, :]
labels = (y_test == 1)[0, :]

print(confusion_matrix(predictions, labels)) 

[[21  0]
 [ 0  9]]


In [49]:
A > 0.5, y_test == 1

(array([[False, False, False, False,  True, False, False, False, False,
          True, False, False, False, False,  True,  True, False, False,
          True, False, False,  True, False,  True, False, False, False,
          True,  True, False]]),
 array([[False, False, False, False,  True, False, False, False, False,
          True, False, False, False, False,  True,  True, False, False,
          True, False, False,  True, False,  True, False, False, False,
          True,  True, False]]))

In [51]:
len((A > 0.5)[0] == (y_test == 1)[0])

30

In [52]:
print(classification_report(predictions, labels))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00        21
        True       1.00      1.00      1.00         9

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [8]:
# My Monthly Budget
revenue = 913890 + 181150 + 1200 * 700
expense = 620000 + 10130 + 35000 + 5000
saving = 500000
my_own_money = revenue - expense - saving
print(f"My monthly free money is {my_own_money}")

My monthly free money is 764910
